<a href="https://colab.research.google.com/github/cindyhfls/SpatialEmbeddedEquilibriumPropagation_Neuromatch_NeuroAI_TrustworthyHeliotrope/blob/main/equilibrium_propagation_toymodel_Tu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted from https://github.com/smonsays/equilibrium-propagation/tree/master "run_energy_model_mnist.py"

**To-do:**

*Week 1 - Make the network architecture and train basic network, decide on the questions*
1. We first make a fake "distance" matrix by specifying the distance between each of the 1000x1000 pairs of units.
2. Implement spatial normalization through energy function?

*Week 2 - Calculating metrics to evaluate the network, each person pick a direction to test and produce a summary slide.*

In [ ]:
# @title Clone Repository and Setup
!git clone https://github.com/cindyhfls/SpatialEmbeddedEquilibriumPropagation_Neuromatch_NeuroAI_TrustworthyHeliotrope.git -b BackPropDev

In [4]:
cd /content/SpatialEmbeddedEquilibriumPropagation_Neuromatch_NeuroAI_TrustworthyHeliotrope/equilibrium-propagation-master/

/content/SpatialEmbeddedEquilibriumPropagation_Neuromatch_NeuroAI_TrustworthyHeliotrope/equilibrium-propagation-master


In [5]:
import argparse
import json
import logging
import sys

import torch

from lib import config, data, energy, train, utils

In [6]:
# @title Install torchlens and other utilities for visualization/RSA?
!pip install torchlens --quiet
!pip install rsatoolbox --quiet
!pip install torchviz --quiet

import torchlens,rsatoolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.8 MB/s eta 0:00:00


In [11]:
# @title Helper functions for parsing input
def load_default_config(energy):
    """
    Load default parameter configuration from file.

    Args:
        tasks: String with the energy name

    Returns:
        Dictionary of default parameters for the given energy
    """
    if energy == "restr_hopfield":
        default_config = "etc/energy_restr_hopfield.json"
    elif energy == "cond_gaussian":
        default_config = "etc/energy_cond_gaussian.json"
    else:
        raise ValueError("Energy based model \"{}\" not defined.".format(energy))

    with open(default_config) as config_json_file:
        cfg = json.load(config_json_file)

    return cfg


def parse_shell_args(args):
    """
    Parse shell arguments for this script.

    Args:
        args: List of shell arguments

    Returns:
        Dictionary of shell arguments
    """
    parser = argparse.ArgumentParser(
        description="Train an energy-based model on MNIST using Equilibrium Propagation."
    )

    parser.add_argument("--batch_size", type=int, default=argparse.SUPPRESS,
                        help="Size of mini batches during training.")
    parser.add_argument("--c_energy", choices=["cross_entropy", "squared_error"],
                        default=argparse.SUPPRESS, help="Supervised learning cost function.")
    parser.add_argument("--dimensions", type=int, nargs="+",
                        default=argparse.SUPPRESS, help="Dimensions of the neural network.")
    parser.add_argument("--energy", choices=["cond_gaussian", "restr_hopfield"],
                        default="cond_gaussian", help="Type of energy-based model.")
    parser.add_argument("--epochs", type=int, default=argparse.SUPPRESS,
                        help="Number of epochs to train.")
    parser.add_argument("--fast_ff_init", action='store_true', default=argparse.SUPPRESS,
                        help="Flag to enable fast feedforward initialization.")
    parser.add_argument("--learning_rate", type=float, default=argparse.SUPPRESS,
                        help="Learning rate of the optimizer.")
    parser.add_argument("--log_dir", type=str, default="",
                        help="Subdirectory within ./log/ to store logs.")
    parser.add_argument("--nonlinearity", choices=["leaky_relu", "relu", "sigmoid", "tanh"],
                        default=argparse.SUPPRESS, help="Nonlinearity between network layers.")
    parser.add_argument("--optimizer", choices=["adam", "adagrad", "sgd"],
                        default=argparse.SUPPRESS, help="Optimizer used to train the model.")
    parser.add_argument("--seed", type=int, default=argparse.SUPPRESS,
                        help="Random seed for pytorch")

    return vars(parser.parse_args(args))

In [12]:
# change this input to make configuration

sys.argv = ['','--energy', 'restr_hopfield', '--epochs', '1']

# Parse shell arguments as input configuration
user_config = parse_shell_args(sys.argv[1:])

# Load default parameter configuration from file for the specified energy-based model
cfg = load_default_config(user_config["energy"])

# Overwrite default parameters with user configuration where applicable
cfg.update(user_config)

# Setup global logger and logging directory
config.setup_logging(cfg["energy"] + "_" + cfg["c_energy"] + "_" + cfg["dataset"],
                      dir=cfg['log_dir'])

In [ ]:
# @title Load data

# Create torch data loaders with the MNIST data set
data_train, data_test = data.create_mnist_loaders(cfg['batch_size'])


In [13]:
# @title Main function run_energy_model_mnist

"""
Main script.

Args:
    cfg: Dictionary defining parameters of the run
"""

# Initialize seed if specified (might slow down the model)
if cfg['seed'] is not None:
    torch.manual_seed(cfg['seed'])

# Create the cost function to be optimized by the model
c_energy = utils.create_cost(cfg['c_energy'], cfg['beta'])

# Create activation functions for every layer as a list
phi = utils.create_activations(cfg['nonlinearity'], len(cfg['dimensions']))

# Initialize energy based model
if cfg["energy"] == "restr_hopfield":
    model = energy.RestrictedHopfield(
        cfg['dimensions'], c_energy, cfg['batch_size'], phi).to(config.device)
elif cfg["energy"] == "cond_gaussian":
    model = energy.ConditionalGaussian(
        cfg['dimensions'], c_energy, cfg['batch_size'], phi).to(config.device)
else:
    raise ValueError(f'Energy based model \"{cfg["energy"]}\" not defined.')

# Define optimizer (may include l2 regularization via weight_decay)
w_optimizer = utils.create_optimizer(model, cfg['optimizer'],  lr=cfg['learning_rate'])

logging.info("Start training with parametrization:\n{}".format(
    json.dumps(cfg, indent=4, sort_keys=True)))

for epoch in range(1, cfg['epochs'] + 1):
    # Training
    train.train(model, data_train, cfg['dynamics'], w_optimizer, cfg["fast_ff_init"])

    # Testing
    test_acc, test_energy = train.test(model, data_test, cfg['dynamics'], cfg["fast_ff_init"])

    # Logging
    logging.info(
        "epoch: {} \t test_acc: {:.4f} \t mean_E: {:.4f}".format(
            epoch, test_acc, test_energy)
    )

In [ ]:
# @title Main function run_backprop_model_mnist (reuse the cfg before for hyperparameters, model architecture etc.)
# Create activation functions for every layer as a list
phi = utils.create_activations(cfg['nonlinearity'], len(cfg['dimensions']))

if cfg['c_energy'] == 'CrossEntropy':
  criterion = nn.functional.CrossEntropyLoss()
elif cft['c_energy'] == 'SquaredLoss':
  criterion = nn.functional.mse_loss()

model = energy.MLP(
    cfg['dimensions'], criterion,cfg['batch_size'],phi).to(config.device)

In [1]:
torch.nn.functiona.mse_loss()

NameError: name 'torch' is not defined

In [14]:
# @title Visualize model
print(model)
import graphviz

def visualize_hopfield_structure(model):
    dot = graphviz.Digraph()
    layers = len(model.W)  # Assuming model.W contains weights between layers

    # Add nodes for each layer
    for i in range(layers + 1):  # +1 because there are n+1 layers if there are n sets of weights
        dot.node(f'Layer {i}', f'Layer {i}')

    # Add edges between nodes
    for i in range(layers):
        dot.edge(f'Layer {i}', f'Layer {i + 1}', label=f'Weights {i}')

    return dot

# Assuming 'model' is an instance of RestrictedHopfield
model_dot = visualize_hopfield_structure(model)
model_dot.render('hopfield_structure', format='png', view=True)


RestrictedHopfield(
  (W): ModuleList(
    (0): Linear(in_features=784, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=10, bias=True)
  )
)


'hopfield_structure.png'

In [ ]:
import graphviz

def visualize_hopfield_connections(model):
    dot = graphviz.Digraph(engine='dot', format='png')

    # Assuming model has an attribute 'W' that represents the weights between layers
    # and 'u' that might represent activations or states of each layer
    layers = len(model.W)  # Number of weight matrices should indicate the number of connections
    units_per_layer = [model.u[i].shape[1] for i in range(layers + 1)]  # +1 to include output layer units

    # Create subgraphs for layers
    for i in range(layers + 1):
        with dot.subgraph(name=f'cluster_{i}') as c:
            c.attr(label=f'Layer {i}')
            for j in range(units_per_layer[i]):
                c.node(f'n_{i}_{j}', f'Unit {j}')

    # Connect units between layers
    for i in range(layers):
        for j in range(model.W[i].weight.shape[0]):  # Rows in the weight matrix for layer i
            for k in range(model.W[i].weight.shape[1]):  # Columns in the weight matrix for layer i
                weight = model.W[i].weight[j, k].item()  # Getting the weight from PyTorch model
                dot.edge(f'n_{i}_{j}', f'n_{i+1}_{k}', label=f'{weight:.2f}')

    return dot

# Create a visualization of the model structure
# Assuming 'model' is properly defined with weights 'W' and states 'u'
model_dot = visualize_hopfield_connections(model)
model_dot.render('hopfield_network', view=True)

In [2]:
model

NameError: name 'model' is not defined

In [26]:
cfg

{'batch_size': 100,
 'beta': 1,
 'c_energy': 'squared_error',
 'dataset': 'mnist',
 'dimensions': [784, 1000, 10],
 'dynamics': {'dt': 0.1, 'n_relax': 50, 'tau': 1, 'tol': 0},
 'energy': 'restr_hopfield',
 'epochs': 1,
 'fast_ff_init': False,
 'learning_rate': 0.001,
 'nonlinearity': 'sigmoid',
 'optimizer': 'adam',
 'seed': None,
 'log_dir': ''}

In [ ]:
# can't get this to work

from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('log/example')
writer.add_graph(model)
writer.close()



!tensorboard --logdir=log

In [7]:
def extract_features(model, imgs, return_layers, plot='none'):
    """
    Extracts features from specified layers of the model.

    Inputs:
    - model (torch.nn.Module): The model from which to extract features.
    - imgs (torch.Tensor): Batch of input images.
    - return_layers (list): List of layer names from which to extract features.
    - plot (str): Option to plot the features. Default is 'none'.

    Outputs:
    - model_features (dict): A dictionary with layer names as keys and extracted features as values.
    """
    model_history = tl.log_forward_pass(model, imgs, layers_to_save='all', vis_opt=plot)
    model_features = {}
    for layer in return_layers:
        model_features[layer] = model_history[layer].tensor_contents.flatten(1)

    return model_features